In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from scipy.stats import linregress

In [2]:
df_stock_price = pd.read_csv('stock_prices.csv',index_col='date',parse_dates=True)
df_stock_price

,ADN,AEE,AEP,AES,AGR,ALCE,ALE,AMPS,ARIS,ARTNA,...,TVE,UGI,UTL,VGAS,VIA,VST,WEC,WTRG,XEL,YORW
date,,,,,,,,,,,,,,,,,,,,,
2023-03-01,52.200001,77.833527,82.796310,23.525192,36.142548,10.200,58.352753,6.63,12.934539,53.065838,...,20.497381,34.530087,51.602707,10.200,29.049999,21.039583,82.554329,41.228195,60.526001,41.644726
2023-03-02,51.599998,79.110909,84.359062,23.534746,36.787613,10.205,58.724483,6.64,13.258378,53.962120,...,20.390877,34.753704,52.164341,9.080,28.400000,20.632555,83.937073,41.170086,61.549595,42.036217
2023-03-03,53.400002,80.417099,85.413658,23.974112,37.280895,10.210,59.334507,6.96,13.115508,53.923149,...,20.671663,35.033226,52.755028,8.990,27.350000,20.748846,85.434242,40.685844,62.276634,42.496220
2023-03-06,52.200001,80.532349,85.682106,23.878599,37.413712,10.220,58.915123,7.14,12.610699,53.747795,...,20.671663,34.576675,52.144970,9.000,25.850000,23.627132,85.672638,40.966709,62.228806,42.643028
2023-03-07,51.599998,79.178139,84.972641,23.410582,36.759155,10.230,57.799915,7.31,10.667661,53.708828,...,20.661982,34.353054,51.931942,8.896,24.750000,25.691357,84.747643,40.191921,60.851261,44.365585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,4.980000,74.709999,86.370003,17.201746,37.020000,0.317,59.959999,3.68,14.660000,34.889999,...,21.559969,26.000000,50.799999,4.340,10.900000,70.459999,81.822365,36.540001,55.330002,35.529999
2024-04-25,4.914000,74.760002,86.860001,16.964069,36.549999,0.330,59.570000,3.60,14.650000,34.520000,...,21.480484,25.730000,50.490002,4.290,10.880000,72.480003,82.000603,36.570000,55.009998,35.369999
2024-04-26,4.548000,73.660004,85.260002,17.043295,36.380001,0.344,59.020000,3.62,14.810000,34.500000,...,21.401001,25.379999,49.939999,4.600,10.880000,72.699997,80.693504,36.090000,53.959999,35.369999


In [3]:
df_train = df_stock_price[df_stock_price.index < '2023-12-31']
df_train

,ADN,AEE,AEP,AES,AGR,ALCE,ALE,AMPS,ARIS,ARTNA,...,TVE,UGI,UTL,VGAS,VIA,VST,WEC,WTRG,XEL,YORW
date,,,,,,,,,,,,,,,,,,,,,
2023-03-01,52.200001,77.833527,82.796310,23.525192,36.142548,10.200,58.352753,6.63,12.934539,53.065838,...,20.497381,34.530087,51.602707,10.200,29.049999,21.039583,82.554329,41.228195,60.526001,41.644726
2023-03-02,51.599998,79.110909,84.359062,23.534746,36.787613,10.205,58.724483,6.64,13.258378,53.962120,...,20.390877,34.753704,52.164341,9.080,28.400000,20.632555,83.937073,41.170086,61.549595,42.036217
2023-03-03,53.400002,80.417099,85.413658,23.974112,37.280895,10.210,59.334507,6.96,13.115508,53.923149,...,20.671663,35.033226,52.755028,8.990,27.350000,20.748846,85.434242,40.685844,62.276634,42.496220
2023-03-06,52.200001,80.532349,85.682106,23.878599,37.413712,10.220,58.915123,7.14,12.610699,53.747795,...,20.671663,34.576675,52.144970,9.000,25.850000,23.627132,85.672638,40.966709,62.228806,42.643028
2023-03-07,51.599998,79.178139,84.972641,23.410582,36.759155,10.230,57.799915,7.31,10.667661,53.708828,...,20.661982,34.353054,51.931942,8.896,24.750000,25.691357,84.747643,40.191921,60.851261,44.365585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,6.354000,71.414375,79.465294,18.722160,31.887262,5.000,60.869202,6.64,8.305835,42.341656,...,21.622562,24.148851,51.537193,2.230,9.390000,37.889172,81.254288,36.819717,60.886734,38.234615
2023-12-26,6.099000,71.117096,79.811317,18.977016,32.133762,2.100,60.928459,6.88,8.454685,42.530109,...,21.671930,24.493694,52.221313,2.200,9.740000,38.068550,81.940056,37.097301,61.073143,39.065159
2023-12-27,6.972000,71.047737,79.890396,18.898602,31.946423,1.780,60.582783,6.92,8.474531,42.292065,...,21.849648,24.503548,52.578251,2.200,9.440000,38.247932,81.753914,37.077480,60.885166,39.114597


In [4]:
def get_hurst_exponent(time_series, max_lag=20):
    lags = range(2, max_lag)
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]
    # Add a small constant to tau to avoid taking log of zero
    tau = np.array(tau) + 1e-10
    reg = np.polyfit(np.log(lags), np.log(tau), 1)
    return reg[0]  # Hurst exponent is the slope


def calculate_crossovers(spread):
    """Calculate the number of zero-crossovers for a time series spread."""
    mean_line = np.mean(spread)
    crossings = np.where(np.diff(np.signbit(spread - mean_line)))[0]
    return len(crossings)



results = []

for i in range(len(df_train.columns)):
    for j in range(i + 1, len(df_train.columns)):
        stock1 = df_train.columns[i]
        stock2 = df_train.columns[j]

        # Apply OLS regression on log-transformed data to get hedge ratio
        model = LinearRegression().fit(np.log(df_train[[stock1]]), np.log(df_train[[stock2]]))
        hedge_ratio = float(model.coef_)

        # Calculate the spread using the hedge ratio
        spread = np.log(df_train[stock2]) - hedge_ratio * np.log(df_train[stock1])

        coint_t, pvalue, _ = coint(df_train[stock1], df_train[stock2])
        
        for lag in [20]:
            hurst_exp = get_hurst_exponent(spread.values, lag)
        
        # Half-life of the spread determined in days
        z_lag = np.roll(spread,1)
        z_lag[0] = 0
        z_ret = spread - z_lag
        z_ret[0] = 0

        # Use a function to determine the half-life in days 
        z_lag2 = sm.add_constant(z_lag)

        model = sm.OLS(z_ret,z_lag2)
        res = model.fit()

        half_life = -np.log(2) / res.params[1] #in days
        
        crossovers = calculate_crossovers(spread)
        
        results.append((stock1, stock2, coint_t, pvalue, hedge_ratio, hurst_exp, half_life, crossovers))

results_df = pd.DataFrame(results, columns=['pairs1', 'pairs2', 'coint_t', 'pvalue', 'hedge_ratio', 'hurst_exponent', 'half_life', 'crossovers'])
results_df

,pairs1,pairs2,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers
0,ADN,AEE,-4.095396,0.005225,0.081478,0.406486,451.894081,16
1,ADN,AEP,-2.933093,0.126978,0.092076,0.483368,801.992693,15
2,ADN,AES,-2.579052,0.245359,0.267413,0.629994,231.566623,7
3,ADN,AGR,-3.471468,0.035010,0.159752,0.476953,211.728380,21
4,ADN,ALCE,-3.631565,0.022382,0.309270,0.323144,-15.762227,10
...,...,...,...,...,...,...,...,...
3316,WEC,XEL,-1.835075,0.612534,1.028897,0.526995,65.236380,10
3317,WEC,YORW,-3.421139,0.040056,1.078228,0.447173,47.516973,23
3318,WTRG,XEL,-2.069520,0.492305,0.562226,0.507829,343.854139,11
3319,WTRG,YORW,-3.955686,0.008297,0.717173,0.429671,74.315450,28


In [9]:
results_df['Pair'] = results_df['pairs1'] +'_' + results_df['pairs2']
results_df.drop(['pairs1', 'pairs2'], axis=1, inplace=True)
results_df

,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers,Pair
0,-4.095396,0.005225,0.081478,0.406486,451.894081,16,ADN_AEE
1,-2.933093,0.126978,0.092076,0.483368,801.992693,15,ADN_AEP
2,-2.579052,0.245359,0.267413,0.629994,231.566623,7,ADN_AES
3,-3.471468,0.035010,0.159752,0.476953,211.728380,21,ADN_AGR
4,-3.631565,0.022382,0.309270,0.323144,-15.762227,10,ADN_ALCE
...,...,...,...,...,...,...,...
3316,-1.835075,0.612534,1.028897,0.526995,65.236380,10,WEC_XEL
3317,-3.421139,0.040056,1.078228,0.447173,47.516973,23,WEC_YORW
3318,-2.069520,0.492305,0.562226,0.507829,343.854139,11,WTRG_XEL
3319,-3.955686,0.008297,0.717173,0.429671,74.315450,28,WTRG_YORW


In [11]:
results_df.to_csv('results.csv')

In [12]:
df_cointegrated = results_df[results_df['pvalue']<0.05]
df_hurst_exponent = df_cointegrated[df_cointegrated['hurst_exponent']<0.5]
df_half_life = df_hurst_exponent[(df_hurst_exponent['half_life'] > 1) & (df_hurst_exponent['half_life'] < 365)]
df_half_life

,coint_t,pvalue,hedge_ratio,hurst_exponent,half_life,crossovers,Pair
3,-3.471468,0.035010,0.159752,0.476953,211.728380,21,ADN_AGR
13,-4.031608,0.006470,0.160480,0.498303,128.708203,9,ADN_BEPC
16,-3.986240,0.007512,0.286938,0.384031,10.659137,17,ADN_CDZI
17,-3.430575,0.039066,-0.311334,0.469681,261.929260,21,ADN_CEG
35,-3.497491,0.032618,-0.462547,0.447678,65.773530,18,ADN_GNE
...,...,...,...,...,...,...,...
3298,-3.431832,0.038936,0.718398,0.360882,97.530674,20,UTL_XEL
3308,-3.444605,0.037632,0.146167,0.489150,347.047706,8,VIA_WTRG
3315,-4.236542,0.003209,1.513684,0.415290,298.155065,34,WEC_WTRG
3317,-3.421139,0.040056,1.078228,0.447173,47.516973,23,WEC_YORW


In [16]:
df_half_life.to_csv('df_half_life.csv',index=False)